In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

# NAME = "Fire-detection-cnn-64x2".format(int(time.time()))
# tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [11]:
# Set a specific fraction of GPU memory to be allocated
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
        print(e)

In [12]:
X=X/255.0
dense_layers=[0]
layer_sizes=[64]
conv_layers=[3]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "Fire-detection-cnn-64x2".format(int(time.time()))            

            model=Sequential()
            model.add(Conv2D(layer_size,(3,3),input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

            model.compile(loss="binary_crossentropy",
                          optimizer="adam",
                          metrics=['accuracy'])
            model.fit(X,y,batch_size=32,epochs=13, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/13
29/29 [==============================] - 4s 136ms/step - loss: 0.5749 - accuracy: 0.7383 - val_loss: 0.6878 - val_accuracy: 0.7200
Epoch 2/13
29/29 [==============================] - 4s 132ms/step - loss: 0.4859 - accuracy: 0.7673 - val_loss: 0.4419 - val_accuracy: 0.8200
Epoch 3/13
29/29 [==============================] - 4s 133ms/step - loss: 0.3781 - accuracy: 0.8352 - val_loss: 0.4496 - val_accuracy: 0.7900
Epoch 4/13
29/29 [==============================] - 4s 130ms/step - loss: 0.3254 - accuracy: 0.8664 - val_loss: 0.3775 - val_accuracy: 0.8700
Epoch 5/13
29/29 [==============================] - 4s 132ms/step - loss: 0.2776 - accuracy: 0.8842 - val_loss: 0.3691 - val_accuracy: 0.8700
Epoch 6/13
29/29 [==============================] - 4s 137ms/step - loss: 0.2686 - accuracy: 0.9009 - val_loss: 0.3444 - val_accuracy: 0.8600
Epoch 7/13
29/29 [==============================] - 4s 126ms/step - loss: 0.2810 - accuracy: 0.8719 - val_loss: 0.4374 - val_accuracy: 0.8300
Epoch 

In [13]:
model.save("fireDetection_64x3-CNN.model")

INFO:tensorflow:Assets written to: fireDetection_64x3-CNN.model/assets
